In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [ ]:
# movielens dataset
names = ['user_id', 'item_id', 'rating', 'timestamp']
data_dir = '../../data/MovieLens/ml-100k'

In [ ]:
data = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names, engine='python')

In [3]:
x = ()
x += (3,)
x

(3,)